In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [8]:
df=pd.read_csv('processed_outliers.csv')
print('Data shape:',df.shape)

Data shape: (1159, 22)


In [9]:
list(enumerate(df.columns))

[(0, 'w/b'),
 (1, 'Water'),
 (2, 'Cement type'),
 (3, 'Cement'),
 (4, 'Slag'),
 (5, 'Fly ash'),
 (6, 'Silica fume'),
 (7, 'Lime filler'),
 (8, 'FA'),
 (9, 'CA'),
 (10, 'Plasticizer'),
 (11, 'Superplasticizer'),
 (12, 'Air entraining'),
 (13, 'Comp. str. test age'),
 (14, 'Compressive strength'),
 (15, 'Air content'),
 (16, 'Spreed'),
 (17, 'Slump'),
 (18, 'Fresh density'),
 (19, 'Dry  density'),
 (20, 'Migration test age'),
 (21, 'Migration resistance')]

1-hot encoding

In [10]:
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

(1159, 30)
(1159,)


XGB

In [11]:
param_grid={
    'n_estimators': [None,10,50,200,300],   #default=100
    'max_depth': [None,2,6,10,20],
    # 'max_leaves': [None,10,20],
    'learning_rate': [None,.05,.1,.5,1],
    'subsample': [None,.5,.6,.7,.8,.9],
    # 'reg_lambda': [None,.1,1,10,100],
    # 'reg_alpha': [None,.01,.1,1],
    # 'colsample_bytree': [None,.2,.5,.8],
    # 'gamma': [None,.1,.5,1],
    # 'min_child_weight': [None,.1,.5,2,4]    #default=1
}

splits=list(StratifiedKFold(shuffle=True,random_state=0).split(X,y))
train_index, test_index = splits[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_xgb(X_train,y_train):
    clf = XGBClassifier(random_state=0,n_jobs=-1)
    grid=GridSearchCV(clf,param_grid,cv=5,n_jobs=3,scoring='accuracy')
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

def cv_scores(X,y,model):
    acc_test=[]
    acc_train=[]
    for train_index, test_index in StratifiedKFold(shuffle=True,random_state=1).split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        model.fit(X_train,y_train)
        acc_test.append(accuracy_score(y_test,model.predict(X_test)))
        acc_train.append(accuracy_score(y_train,model.predict(X_train)))
    print('Train acc:',np.mean(acc_train))
    print('Test acc:',np.mean(acc_test))

In [12]:
grid=train_xgb(X_train,y_train)

{'learning_rate': None, 'max_depth': 20, 'n_estimators': 300, 'subsample': 0.5}
0.7714850334205173


In [13]:
best_model=XGBClassifier(random_state=0,n_jobs=-1,**grid.best_params_)
best_model.fit(X_train,y_train)
accuracy_score(y_test,best_model.predict(X_test))

0.853448275862069

In [14]:
cv_scores(X,y,best_model)

Train acc: 1.0
Test acc: 0.8576541274817137


Removing cement type

In [15]:
X=df.drop('Cement type',axis=1).iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(1159, 20)
(1159,)


In [16]:
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
grid=train_xgb(X_train,y_train)

{'learning_rate': 1, 'max_depth': None, 'n_estimators': None, 'subsample': None}
0.7768497529787852


In [17]:
best_model=XGBClassifier(random_state=0,n_jobs=-1,**grid.best_params_)
best_model.fit(X_train,y_train)
accuracy_score(y_test,best_model.predict(X_test))

0.8706896551724138

In [18]:
cv_scores(X,y,best_model)

Train acc: 1.0
Test acc: 0.8524705179877593
